In [2]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pprint import pprint
from IPython.display import display
from tqdm import tqdm
import glob

%load_ext autoreload
%autoreload 
%matplotlib inline


# sys.path.append(os.path.dirname(os.getcwd()))
# from src import utils

## TEST

In [3]:
import requests

url = "http://127.0.0.1:8800"
response = requests.get(f"{url}/env")
pprint(response.json())

{'objects': {'object_red_0': {'id': 208,
                              'ori': [6.232128890690638e-17,
                                      0.0,
                                      2.3876785173585704e-18],
                              'pos': [-0.2999999999988103,
                                      -0.6499999999995241,
                                      0.9397844891595659]},
             'object_red_bowl_0': {'id': 210,
                                   'ori': [-3.771517215122816e-07,
                                           1.4758042643503444e-06,
                                           -9.723745469803667e-07],
                                   'pos': [-2.200000116058312,
                                           -0.04999979322137696,
                                           0.9624647301876156]},
             'object_yellow_0': {'id': 209,
                                 'ori': [7.886205589765255e-17,
                                         0.0,
                   

## Planning

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
# use the already-imported src.utils (imported in an earlier cell)
from agent.src.config.config_decomp import load_config

config = load_config("./agent/src/config/config_decomp.yaml")
print(config)

paths=PathsConfig(output_dir='output/', prompt_dir='src/graph/prompts/') runner=RunnerConfig(goal_decomp_node=NodeConfig(model_name='gpt41mini', prompt_cache_key='goal_decomp_node'), task_decomp_node=NodeConfig(model_name='gpt41mini', prompt_cache_key='task_decomp_node')) skills=[RobotSkillConfig(name='robot1', skills=['GoToObject', 'PickObject', 'PlaceObject'])]


In [6]:
from agent.src.runner.state import BaseStateMaker


state_maker = BaseStateMaker(config)
state = state_maker.make(
    user_query="Organize the objects to the bowls according to their colors"
)

pprint(state)

Making inputs for state...
{'object_red_0': {'id': 208, 'pos': [-0.2999999999988103, -0.6499999999995241, 0.9397844891595659], 'ori': [3.8091214264397464e-17, 0.0, 2.3876785343807713e-18]}, 'object_yellow_0': {'id': 209, 'pos': [0.4000000000008829, 0.6500000000008823, 0.9397844891595659], 'ori': [7.280560477478075e-17, 0.0, -5.584044970393158e-20]}, 'object_red_bowl_0': {'id': 210, 'pos': [-2.2000000522185528, -0.04999971211677102, 0.9624647292493889], 'ori': [-2.284598050832429e-06, 2.981804149593259e-06, -9.73441452355865e-07]}, 'object_yellow_bowl_7': {'id': 211, 'pos': [-0.30002481115490337, 0.49996658900404295, 0.9696173816803142], 'ori': [0.0006989506600626067, -0.0005170940877630148, -4.343160051369912e-07]}}
url: http://127.0.0.1:8800
{'inputs': {'object_text': '{{\n'
                           '"object_name": "object_red_0",\n'
                           '"object_name": "object_yellow_0",\n'
                           '"object_name": "object_red_bowl_0",\n'
                   

In [9]:
import time
from agent.src.runner.runner import DecompRunner

runner = DecompRunner(config=config)

# Add a small delay if you're making multiple rapid requests
# time.sleep(1)

final_state = runner.invoke(state)

2025-12-08 21:01:57,072 [INFO] agent.src.runner.graph: ============= GOAL_DECOMP_NODE ==============
2025-12-08 21:01:58,935 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 21:01:58,948 [INFO] agent.src.runner.graph: AI Answer:
{'subgoals': ['Organize the red objects to the red bowls', 'Organize the yellow objects to the yellow bowls']}

2025-12-08 21:01:58,954 [INFO] agent.src.runner.graph: ============= TASK_DECOMP_NODE ==============


Subgoals Text:
[
1. Organize the red objects to the red bowls
2. Organize the yellow objects to the yellow bowls
]



2025-12-08 21:02:02,006 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 21:02:02,020 [INFO] agent.src.runner.graph: AI Answer:
{'task_outputs': [{'subgoal': 'Organize the red objects to the red bowls', 'tasks': [{'skill': 'GoToObject', 'target': 'object_red_0'}, {'skill': 'PickObject', 'target': 'object_red_0'}, {'skill': 'GoToObject', 'target': 'object_red_bowl_0'}, {'skill': 'PlaceObject', 'target': 'object_red_bowl_0'}]}, {'subgoal': 'Organize the yellow objects to the yellow bowls', 'tasks': [{'skill': 'GoToObject', 'target': 'object_yellow_0'}, {'skill': 'PickObject', 'target': 'object_yellow_0'}, {'skill': 'GoToObject', 'target': 'object_yellow_bowl_7'}, {'skill': 'PlaceObject', 'target': 'object_yellow_bowl_7'}]}]}



In [10]:
task_outputs = final_state["tasks"]["task_outputs"]
pprint(task_outputs)
print(type(task_outputs))

[{'subgoal': 'Organize the red objects to the red bowls',
  'tasks': [{'skill': 'GoToObject', 'target': 'object_red_0'},
            {'skill': 'PickObject', 'target': 'object_red_0'},
            {'skill': 'GoToObject', 'target': 'object_red_bowl_0'},
            {'skill': 'PlaceObject', 'target': 'object_red_bowl_0'}]},
 {'subgoal': 'Organize the yellow objects to the yellow bowls',
  'tasks': [{'skill': 'GoToObject', 'target': 'object_yellow_0'},
            {'skill': 'PickObject', 'target': 'object_yellow_0'},
            {'skill': 'GoToObject', 'target': 'object_yellow_bowl_7'},
            {'skill': 'PlaceObject', 'target': 'object_yellow_bowl_7'}]}]
<class 'list'>


In [12]:
# graph = runner.graph

# graph_image = graph.get_graph().draw_mermaid_png()
# with open("graph.png", "wb") as f:
#     f.write(graph_image)

## apply

In [13]:
import requests

url = "http://127.0.0.1:8800"
response = requests.get(f"{url}/env")
object_map = response.json().get("objects", [])

In [14]:
pprint(object_map)
pprint(task_outputs)

{'object_red_0': {'id': 208,
                  'ori': [8.008740528772158e-17, 0.0, 2.4357430824515283e-18],
                  'pos': [-0.2999999999988103,
                          -0.6499999999995241,
                          0.9397844891595659]},
 'object_red_bowl_0': {'id': 210,
                       'ori': [-3.0761307221276584e-06,
                               9.750340383973821e-07,
                               -9.867313775543744e-07],
                       'pos': [-2.2000001399101587,
                               -0.04999967641096279,
                               0.9624645755218433]},
 'object_yellow_0': {'id': 209,
                     'ori': [7.30177345068181e-17, 0.0, -5.584044978513495e-20],
                     'pos': [0.4000000000008829,
                             0.6500000000008823,
                             0.9397844891595659]},
 'object_yellow_bowl_7': {'id': 211,
                          'ori': [0.0006989506600626281,
                                  -0

In [ ]:
from agent.src.runner.executor import TaskExecutor

task_executor = TaskExecutor()
task_executor.execute(task_outputs)

[{'skill': 'GoToObject', 'target': 'object_red_0'},
 {'skill': 'PickObject', 'target': 'object_red_0'},
 {'skill': 'GoToObject', 'target': 'object_red_bowl_0'},
 {'skill': 'PlaceObject', 'target': 'object_red_bowl_0'},
 {'skill': 'GoToObject', 'target': 'object_yellow_0'},
 {'skill': 'PickObject', 'target': 'object_yellow_0'},
 {'skill': 'GoToObject', 'target': 'object_yellow_bowl_7'},
 {'skill': 'PlaceObject', 'target': 'object_yellow_bowl_7'}]
{}
{}
{}
{}
{}
{}
{}
{}
